In [13]:
import geopandas as gpd
import pandas as pd
import requests
import os
import shutil
import getpass
import glob
import json
import duckdb
import io
import getpass

In [26]:
http_proxy = getpass.getpass(prompt='http_proxy')
https_proxy = getpass.getpass(prompt='https_proxy')
proxies = { "http"  :   http_proxy, "https" :  https_proxy}

http_proxy········
https_proxy········


In [9]:
duckdb.sql('LOAD SPATIAL;')

In [30]:
duckdb.sql("SET http_proxy = '"+ http_proxy + "';")

In [27]:
s = requests.get('https://www.data.gouv.fr/fr/datasets/r/721bdff1-14b0-4fa5-a949-bd12f9ce714b', proxies=proxies).text

liste_file = pd.read_json(io.StringIO(s))

In [134]:
liste_file_93 = liste_file[liste_file.proj=='lamb93']
liste_file_93 = liste_file_93.sort_values(['dep', 'year'])
liste_file_93 = liste_file_93.groupby(['dep']).last().reset_index()

In [ ]:
for index, row in liste_file_93.iterrows():
    print(row.dep)
    duckdb.sql("CREATE OR REPLACE VIEW ocs_ge AS from read_parquet('{}')".format(row['url']))
    duckdb.sql("create or replace table occ_sol as select *  from ocs_ge;")
    duckdb.sql("create or replace table occ_sol as " 
           "select *, ST_Transform (geometry, 'EPSG:2154', 'EPSG:3035') "
           "from occ_sol")
    duckdb.sql("create index sindex on occ_sol USING RTREE(geometry);")
    duckdb.sql("create or replace table etendue as "  
           "select st_Xmin(geometry) as xmin, st_Xmax(geometry) as xmax, st_Ymin(geometry) as ymin, st_Ymax(geometry) as ymax "
           "from (select ST_Extent_Agg(geometry) as geometry from occ_sol);")
    duckdb.sql("set variable size = 1000;"
           "create or replace table carreaux_size as " 
           "select x,y, ST_GeomFromText('POLYGON(('||x||' '||y||',"
                                       "'||x + getvariable('size')||' '||y||', "  
                                       "'||x + getvariable('size')||' '||y+getvariable('size')||', "
                                       "'||x||' '||y+getvariable('size')||', "
                                       "'||x||' '||y||'))') as geometry "
           "from "
           "(select unnest(generate_series(cast(floor(xmin/getvariable('size'))*getvariable('size') as int), cast(floor(xmax/getvariable('size'))*getvariable('size') as int), getvariable('size'))) as x from etendue), "
           "(select unnest(generate_series(cast(floor(ymin/getvariable('size'))*getvariable('size') as int), cast(floor(ymax/getvariable('size'))*getvariable('size') as int), getvariable('size'))) as y from etendue);")
    duckdb.sql("create or replace table occupation_carroye as " 
           "select x, y, CODE_CS, sum(st_area(st_intersection(occ_sol.geometry, carreaux_size.geometry))) as surface "
           "from occ_sol, carreaux_size " 
           "where ST_Intersects(occ_sol.geometry, carreaux_size.geometry) "
           "group by x,y,CODE_CS;")
    duckdb.sql("COPY occupation_carroye TO carreaux_"+ row.dep + "_"+ str(row.year)  +".csv ")

d001


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

d002


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

d022


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [129]:
first_rows = liste_file_93.groupby(['dep']).first().reset_index()

In [130]:
first_rows

,dep,id,name,url,proj,year
0,d001,db3fc553-44be-4ba0-a4d1-938d0470f05a,ocs-ge-2-0-shp-lamb93-d001-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
1,d002,d4c58925-ce60-431a-a0f3-f4cc969853ba,ocs-ge-2-0-shp-lamb93-d002-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
2,d004,332103e4-f349-438f-8642-6aa07c6719db,ocs-ge-2-0-shp-lamb93-d004-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
3,d005,c0b1d8d3-fbb2-417f-afea-818f7a4c3d6f,ocs-ge-2-0-shp-lamb93-d005-2022-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2022
4,d006,da14c57e-f3d5-4ff7-bf8c-3358a93f1128,ocs-ge-2-0-shp-lamb93-d006-2020-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2020
...,...,...,...,...,...,...
58,d091,47877895-914d-4871-996a-f2bbf9bb9767,ocs-ge-2-0-shp-lamb93-d091-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
59,d092,ea38e65c-02a4-43dd-8fe4-4f3c89f7a221,ocs-ge-2-0-shp-lamb93-d092-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
60,d093,a674e8db-9cb8-4d33-9e4e-552b7def1c21,ocs-ge-2-0-shp-lamb93-d093-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
61,d094,d79d16d0-bd15-4a04-8e25-6abcac552da2,ocs-ge-2-0-shp-lamb93-d094-2021-01-01.parquet,https://static.data.gouv.fr/resources/occupati...,lamb93,2021
